# Importación de librerías necesarias

In [12]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import StratifiedShuffleSplit

# Definición del modelo

In [13]:
class AffectNetCNN(nn.Module):
    def __init__(self, num_classes=2, lr=0.001):
        super(AffectNetCNN, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.conv1_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv2_layer = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        self.optimizer = optim.AdamW(self.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=2
        )
        self.to(self.device)

    def forward(self, x):
        x = x.to(self.device)
        x = self.conv1_layer(x)
        x = self.conv2_layer(x)
        x = torch.flatten(x, 1)
        x = self.fc_layer(x)
        return x

    def _loss_(self, x, target, class_weights=None):
        x, target = x.to(self.device), target.to(self.device)
        if class_weights is not None:
            class_weights = class_weights.to(self.device)
            loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        else:
            loss_fn = nn.CrossEntropyLoss()
        return loss_fn(x, target)

    def train_(self, data_loader, epochs=10, class_weights=None):
        for epoch in range(epochs):
            total_loss = 0
            for x, target in data_loader:
                x, target = x.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()
                loss = self._loss_(self.forward(x), target, class_weights)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            self.scheduler.step(total_loss / len(data_loader))
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data_loader)}')

    def test_(self, data_loader):
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, target in data_loader:
                x, target = x.to(self.device), target.to(self.device)
                output = self.forward(x)
                _, predicted = torch.max(output, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        return correct / total

# Definición del transformador de las imágenes del dataset

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Carga del conjunto de datos

In [15]:
path = os.getcwd()
dataset = datasets.ImageFolder(os.path.join(path, 'AffectNet'), transform=transform)
num_classes = len(dataset.classes)
print('Dataset classes:', dataset.classes)

from collections import Counter
class_counts = Counter(dataset.targets)
total_samples = sum(class_counts.values())
num_classes = len(class_counts)
class_weights = [total_samples / (num_classes * count) for count in class_counts.values()]
class_weights = torch.tensor(class_weights, dtype=torch.float32)

print("Class counts:", class_counts)
print("Weights per class:", class_weights)

Dataset classes: ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Class counts: Counter({5: 1880, 4: 1862, 7: 1851, 3: 1839, 1: 1833, 0: 1822, 6: 1821, 2: 1740})
Weights per class: tensor([1.0049, 0.9989, 1.0523, 0.9956, 0.9834, 0.9739, 1.0055, 0.9892])


# Obtención de las etiquetas para la estratificación

In [16]:
targets = dataset.targets

# Creación de índices de train y test de manera estratificada

In [17]:
test_size = 0.3 # Porcentaje de datos para el conjunto de prueba
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
train_idx, test_idx = next(stratified_split.split(np.arange(len(targets)), targets))

# Creación de samplers para los DataLoaders

In [18]:
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

# Creación de los DataLoaders con los samplers

In [19]:
train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=64, sampler=test_sampler)

# Inicialización y entrenamiento del modelo

In [20]:
model = AffectNetCNN(num_classes=num_classes, lr=0.0001)

print('Training CNN model...')
model.train_(train_loader, epochs=40, class_weights=class_weights)

Training CNN model...
Epoch 1/40, Loss: 2.076885842388461
Epoch 2/40, Loss: 1.7120356641200758
Epoch 3/40, Loss: 1.5837268177766977
Epoch 4/40, Loss: 1.5176324511166686
Epoch 5/40, Loss: 1.4618888352968678
Epoch 6/40, Loss: 1.428621694908379
Epoch 7/40, Loss: 1.3914653581121694
Epoch 8/40, Loss: 1.3564483608518327
Epoch 9/40, Loss: 1.3218818703053161
Epoch 10/40, Loss: 1.2938289201777915
Epoch 11/40, Loss: 1.249253613608224
Epoch 12/40, Loss: 1.2292819563646493
Epoch 13/40, Loss: 1.193581369352637
Epoch 14/40, Loss: 1.1601481741259556
Epoch 15/40, Loss: 1.1334828390097766
Epoch 16/40, Loss: 1.1029082259035998
Epoch 17/40, Loss: 1.0676490520098194
Epoch 18/40, Loss: 1.0291181236320401
Epoch 19/40, Loss: 0.9882279323494952
Epoch 20/40, Loss: 0.952714875247908
Epoch 21/40, Loss: 0.9454283133056594
Epoch 22/40, Loss: 0.8861193486622402
Epoch 23/40, Loss: 0.8792990901455375
Epoch 24/40, Loss: 0.822005171583306
Epoch 25/40, Loss: 0.8133489937515732
Epoch 26/40, Loss: 0.767491344709574
Epoch 

# Evaluación del modelo en el conjunto de prueba

In [21]:
accuracy = model.test_(test_loader)
print(f'Test Accuracy: {accuracy:.2f}')

Test Accuracy: 0.60


# Guardado del modelo entrenado

In [22]:
torch.save(model.state_dict(), 'model.pth')